# Tools to work with tables extracted from csi softwares ETABS,SAFE,SAP2000

In [4]:
from pathlib import Path
import pandas as pd
import openpyxl
Path.cwd().resolve()

WindowsPath('D:/OneDrive - Hesco/2025/_automation/csi')

In [5]:
# pip install -e .
# pip uninstall mat
import mat
mat.say_hi()

Hello World 2025-03-24 18:14:22


'Hello World 2025-03-24 18:14:22'

In [6]:
folder_in = Path('./sample_data')
folder_out = Path('./sample_output')
folder_in.resolve()
sheet_names = {
    'joints' : 'Objects and Elements - Joints',
    'joint_forces' : 'Element Joint Forces - Shells',
    'areas' : 'Objects and Elements - Areas',
    'area_stresses' : 'Element Stresses - Area Shells',
    'area_forces' : 'Element Forces - Area Shells',
}
sheet_names
xlfile = mat.find_files(folder_in,recursive=True)[0]

In [7]:
joint_df = pd.read_excel(
    xlfile,
    sheet_name = sheet_names['joints'],
    skiprows=[0,2],
)
joint_df = joint_df[['Story','Element Name','Global X','Global Y','Global Z']]
joint_df = joint_df.rename(columns={
    'Element Name':'name',
    'Story' : 'story',
    'Global X' : 'x',
    'Global Y' : 'y',
    'Global Z' : 'z',
})
joint_df.loc[joint_df.story == 'BASE']

,story,name,x,y,z
3000,BASE,24,0.0,10000.0,0
3001,BASE,36,10000.0,10000.0,0
3002,BASE,42,10000.0,0.0,0
3003,BASE,48,0.0,0.0,0
3004,BASE,54,5000.0,5000.0,0
...,...,...,...,...,...
3832,BASE,~3833,10725.2,293.7,0
3833,BASE,~3834,10754.4,10.6,0
3834,BASE,~3835,10554.1,19.4,0
3835,BASE,~3836,10724.8,-259.2,0


In [8]:
area_df = pd.read_excel(
    xlfile,
    sheet_name=sheet_names['areas'],
    skiprows=[0,2]
)
area_df = area_df.drop(columns=['Object Type','Object Label','Object Name'])
area_df = area_df.rename(columns={
    'Story' : 'story',
    'Element Name' : 'name',
    'Elm Jt1' : 'jt1',
    'Elm Jt2' : 'jt2',
    'Elm Jt3' : 'jt3',
    'Elm Jt4' : 'jt4',
})
area_df.loc[area_df.story == 'BASE']

,story,name,jt1,jt2,jt3,jt4
2940,BASE,1-1,36,~3010,~3011,~3012
2941,BASE,1-2,~3013,~3014,~3011,~3010
2942,BASE,1-3,~3014,~3015,~3016,~3011
2943,BASE,1-4,~3015,~3017,~3018,~3016
2944,BASE,1-5,~3017,~3019,~3020,~3018
...,...,...,...,...,...,...
3752,BASE,1-813,~3837,~3831,~3834,~3836
3753,BASE,1-814,~3834,~3833,~3835,NaN
3754,BASE,1-815,~3834,~3835,~3836,NaN
3755,BASE,1-816,~3201,~3202,~3211,NaN
